In [1]:
import numpy as np
from syft.core.tensor.autodp.phi_tensor import PhiTensor
from syft.core.tensor.autodp.gamma_tensor import GammaTensor
from syft.core.tensor.fixed_precision_tensor import FixedPrecisionTensor as FPT
from syft.core.adp.data_subject_list import DataSubjectList
from syft.core.tensor.lazy_repeat_array import lazyrepeatarray as lra

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Torch Dataset equivalent

We have to implement:
- padding -> DONE; needs LRA Implementation
- random horizontal flip
- random vertical flip
- random rotation
- normalize

In [2]:
a = PhiTensor(child=np.random.random((5,5)),data_subjects=np.ones((5,5)), min_vals=0, max_vals=1)

In [3]:
a.pad(width=1)

PhiTensor(child=FixedPrecisionTensor(child=[[55504 63718 55504 27632 52084 51937 52084]
 [33834 36499 33834 22439 30165 62863 30165]
 [55504 63718 55504 27632 52084 51937 52084]
 [53553 60535 53553 12064 17115  5578 17115]
 [29879  4088 29879 58644 34088 28847 34088]
 [21054 48476 21054 21786 11059 17184 11059]
 [29879  4088 29879 58644 34088 28847 34088]]), min_vals=<lazyrepeatarray data: 0 -> shape: (6, 6)>, max_vals=<lazyrepeatarray data: 1 -> shape: (6, 6)>)

In [55]:
d = np.random.random((5,5))
d

array([[0.87432296, 0.20823816, 0.00098661, 0.95585973, 0.61810754],
       [0.00534942, 0.46297501, 0.14584004, 0.12418829, 0.34781126],
       [0.65523659, 0.24482093, 0.9297652 , 0.620728  , 0.15386172],
       [0.22302492, 0.57638697, 0.14881784, 0.77577571, 0.30402905],
       [0.89354963, 0.49164349, 0.55097254, 0.92057209, 0.01183032]])

In [87]:
np.pad(d, pad_width=1, mode="reflect")

array([[0.46297501, 0.00534942, 0.46297501, 0.14584004, 0.12418829,
        0.34781126, 0.12418829],
       [0.20823816, 0.87432296, 0.20823816, 0.00098661, 0.95585973,
        0.61810754, 0.95585973],
       [0.46297501, 0.00534942, 0.46297501, 0.14584004, 0.12418829,
        0.34781126, 0.12418829],
       [0.24482093, 0.65523659, 0.24482093, 0.9297652 , 0.620728  ,
        0.15386172, 0.620728  ],
       [0.57638697, 0.22302492, 0.57638697, 0.14881784, 0.77577571,
        0.30402905, 0.77577571],
       [0.49164349, 0.89354963, 0.49164349, 0.55097254, 0.92057209,
        0.01183032, 0.92057209],
       [0.57638697, 0.22302492, 0.57638697, 0.14881784, 0.77577571,
        0.30402905, 0.77577571]])

In [73]:
output = np.concatenate((d[0:2], d, d[-2:]))

In [74]:
output

array([[0.87432296, 0.20823816, 0.00098661, 0.95585973, 0.61810754],
       [0.00534942, 0.46297501, 0.14584004, 0.12418829, 0.34781126],
       [0.87432296, 0.20823816, 0.00098661, 0.95585973, 0.61810754],
       [0.00534942, 0.46297501, 0.14584004, 0.12418829, 0.34781126],
       [0.65523659, 0.24482093, 0.9297652 , 0.620728  , 0.15386172],
       [0.22302492, 0.57638697, 0.14881784, 0.77577571, 0.30402905],
       [0.89354963, 0.49164349, 0.55097254, 0.92057209, 0.01183032],
       [0.22302492, 0.57638697, 0.14881784, 0.77577571, 0.30402905],
       [0.89354963, 0.49164349, 0.55097254, 0.92057209, 0.01183032]])

In [75]:
output.shape

(9, 5)

In [72]:
d[0:2]

array([[0.87432296, 0.20823816, 0.00098661, 0.95585973, 0.61810754],
       [0.00534942, 0.46297501, 0.14584004, 0.12418829, 0.34781126]])

In [113]:
def pad(dp_tensor: PhiTensor, width: float, padding_mode:str="reflect") -> PhiTensor:
    if padding_mode == "reflect":
        output_data = np.pad(dp_tensor.child.decode(), pad_width=width, mode=padding_mode)
        
        original_indexed = dp_tensor.data_subjects.data_subjects_indexed.copy()
        output_subjects = DataSubjectList(
            one_hot_lookup=dp_tensor.data_subjects.one_hot_lookup,
            data_subjects_indexed=np.pad(original_indexed, pad_width=width, mode=padding_mode)
        )
        
        output_min_vals = dp_tensor.min_vals
        output_max_vals = dp_tensor.max_vals
        
    else:
        raise NotImplementedError
    return PhiTensor(
        child=FPT(output_data),
        data_subjects=output_subjects,
        min_vals=output_min_vals,
        max_vals=output_max_vals
    )

In [115]:
pad(a, width=1)

PhiTensor(child=FixedPrecisionTensor(child=[[59501 19298 59501 57800 17703 41541 17703]
 [35346 11596 35346 39638 23254 23946 23254]
 [59501 19298 59501 57800 17703 41541 17703]
 [63908 54675 63908 12507 28145 20465 28145]
 [53854 50163 53854 39356 58727  4275 58727]
 [29121 54846 29121 45775 58021  5107 58021]
 [53854 50163 53854 39356 58727  4275 58727]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [116]:
a

PhiTensor(child=FixedPrecisionTensor(child=[[11596 35346 39638 23254 23946]
 [19298 59501 57800 17703 41541]
 [54675 63908 12507 28145 20465]
 [50163 53854 39356 58727  4275]
 [54846 29121 45775 58021  5107]]), min_vals=<lazyrepeatarray data: 0 -> shape: (5, 5)>, max_vals=<lazyrepeatarray data: 1 -> shape: (5, 5)>)

In [120]:
laze = lra(data=1, shape=(5,5))

In [124]:
laze.data.size == 1

True

<hr>
<hr>

## Model Training


To Do:
- Implement the layers needed for a ConvNet for DP Tensors:
    - Conv layer
    - BatchNorm2D
    - LeakyReLU
    - AvgPool2d
    - Linear
- Do 1 forward pass with these layers
- Do 1 backprop with these layers

In [ ]:
def conv2D(image, out_channels, kernel_size, padding=0, strides=1):
    pass

In [4]:
import torch

In [5]:
import cv2

In [7]:
img = cv2.imread("/home/e/Downloads/Dataset/10253/0/10253_idx5_x1001_y1001_class0.png")
img = cv2.resize(img, (50, 50))
img.shape

(50, 50, 3)

In [85]:
transforms.Pad(64, padding_mode="reflect")(transforms.ToPILImage()(img)).size

(178, 178)